# TP4 - Background Substraction
ATRIM - Option Datasim

Ecole Centrale Nantes

Diana Mateus

Participants: Yassine JAMOUD, Samy HAFFOUDHI

### BACKGROUND SUBSTRACTION 

The goal of this TP is to enhance the video of a neurointervention  to improve the visualization of moving tools. To this end you will implement a pipeline of image processing methods to detect the moving tools automatically. 


#### Methodology

As the brain is mostly static one way to detect the moving tools is to substract the background (first image) from each of the subsequent images. However, the results of this step need to be further improved. To this end, you will design a pipilene with the methods learnt in this course to produce a binary mask for the pixels that belong to the tools. 

In your pipeline use at least:
- one histogram transformation
- one morphological operation
- one filtering operation in the spatial domain
- one filtering operation in the spectral domain
- one segmentation method

_The same pipeline should be applied to every image_


#### Expected output

The output of your image processing pipeline should be one binary image mask (with values 0 or 1) for every input image of the sequence, where 
- the zero valued pixels indicate the moving tools inside each image.  
- the pixels with value 1 indicate the background (not a moving tool)

To validate the proposed method, a human has annotated (manually drawn) the tools of interest within the images. The annotated pixels belong either to catheters or guidewires. **Your masks should be as close as possible to the human annotations.**


#### Visualization of data and manual annotations

- Data visualization  (**do not include in final version**): visualize the neurointervention images in the ``` catheter``` folder with name ```frame_#```

- Individual Annotation visualization  (** do not include in final version**): visualize the manual annotations in the ```catheter``` folder with names ``` #_MicroCath``` and ```#_GuideWire```. 

- Individual Annotation visualization  (** do not include in final version**): visualize the full manual annotation (union of the guidewire and microcatheter masks) by composing the union of the ``` #_MicroCath``` and ```#_GuideWire```. It should also be a binary mask.



#### Experimental (quantitative and qualitative  validation)

To compare your results and the manual annotations use the mean SAD (Sum of Absolute Differences) and the SNR (Signal to Noise Ratio) errors between your  mask and  the **full** manual mask. 

Present the results qualitatively and quantitatively:

- Qualitatively: 
     - Show your mask side by side with the manually annotated mask
     - Create an enhanced image suitable for guidance: enhance the contrast of the image and overlay your mask on the green channel of the enhanced image.

- Quantitatively: 
    - compute and print the SAD (sum of absolute differences) error per image. 
    - compute and print the MSE (sum of squared  differences ) error per image.
    - compute and print the PSNR (Peak signal to noise ratio) taking as reference image the manual annotations. 
    - Then compute and print the mean and standard deviation of the three measures (SAD, MSE and PSNR) over the entire sequence.
    
Hints:
```
mse = numpy.mean( (img1 - img2) ** 2 )
PIXEL_MAX = 255.0 #or 1.0 or max over the signal of interest
psnr = 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
```
or look at ```skimage.measure``` module

You may use modules such as ```scipy```, ```skimage``` or ``sklearn``(e.g. for clustering with K-means or a Gaussian Mixture Model). Ask me for other external modules.


### REPORT INSTRUCTIONS

#### 1. Intermediate Steps (Code and Description)
Report the results of the intermediate steps (when you add or remove a method from the pipeline):
- provide a text introduction with the idea that you intend to try
- show the implementation of the idea with code 
- evaluate the quantitative and qualitative changes  when including, varying, adapting, etc the proposed method
- Discuss the scores or visualization improvements/degradations 

#### 2. Final Pipeline (Code and Description)
Provide a detailed description of the best performing pipeline. Comment the code such that it is straightforward to relate the pipeline description to the code. Add your conclusions

- Describe the final retained pipeline
- Give a justification for every step (e.g. supported by experimental intermediate steps or theory). 
- Add the **commented** code
- Display the qualitative and quantitative results 
- Give your conclusions

## Présentation de la méthode envisagée

À partir des différences entre les différentes frames et l'image de fond on réalisera :

* un premier thresholding afin de mieux distinguer l'outil du fond noir, la valeur du threshold sera obtenu suite à une visualisation au préalable de l'histogramme
* un filtrage passe-bas afin de réduire le bruit
* une suppression manuelle d'une ligne verticale parasite à gauche des images et hors de la région d'interêt. On peut également utiliser cette méthode pour supprimer une partie gauche de l'outil pas présente sur les images objectifs fournies (cf ligne commentée de la partie "Suppression maneulle")
* une erosion afin de supprimer des points isolés de l'image et affiner le tracé
* un thresholding afin d'obtenir le type de masque desiré

## Réalisation

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import skimage.io as io
from scipy import ndimage
from copy import deepcopy

### Visualisation des frames

In [ ]:
IMDIR = 'catheter'
fig=plt.figure(figsize=(10, 10))

frames = {}

for root, dirnames, filenames in os.walk(IMDIR):
    for filename in filenames:
        f = os.path.join(root, filename)
        if filename.startswith(('frame_')):
            if filename.endswith('201'):
                background = io.imread(f, as_gray=True).astype('float') / 255
            else:
                frames[filename[-3:]] = io.imread(f, as_gray=True).astype('float') / 255
                
i = 1
for key in frames:
    plt.subplot(3, 3, i)
    plt.imshow(frames[key], cmap='gray')
    plt.axis('off')
    plt.title(key)
    i += 1
        
plt.show()

### Visualisation des différences avec le fond

In [ ]:
fig = plt.figure(figsize=(10, 10))

diff = {}
for key in frames:
    diff[key] = abs(frames[key] - background)   
    
i = 1
for key in diff:
    plt.subplot(3, 3, i)
    plt.imshow(diff[key], cmap='gray')
    plt.axis('off')
    plt.title(key)
    i += 1
    
plt.show()

### Visualisation des histogrammes

In [ ]:
fig=plt.figure(figsize=(10, 40))

i = 1
for key in diff:
    hist, _ = np.histogram(diff[key], bins=256)
    plt.subplot(9, 2, i)
    plt.imshow(diff[key], cmap='gray')
    plt.subplot(9, 2, i+1)
    plt.plot(hist)
    i += 2

plt.show()

### Thresholding

In [ ]:
fig=plt.figure(figsize=(10, 10))

diff_threshold = deepcopy(diff)

i = 1
for key in diff:
    diff_threshold[key] = np.where(diff[key] < 0.12 * np.max(diff[key]), 0, 1)
    plt.subplot(3, 3, i)
    plt.imshow(diff_threshold[key], cmap='gray')
    plt.axis('off')
    plt.title(key)
    i += 1

plt.show()

Maintenant nous distinguons mieux l'outil et obtenons un résultat assez satisfaisant par un simple thresholding

### Filtre passe-bas

In [ ]:
def lowpass_filter(D, shape, filter_type='ideal'):
    Q, P = shape
    if filter_type == 'ideal':
        lowpass = np.array([[int((u - P//2)**2 + (v - Q//2)**2 < D**2) for u in range(P)] for v in range(Q)])
    else:
        lowpass = np.array([[ np.exp(-((u - P//2)**2 + (v - Q//2)**2 )/(2*D**2)) for u in range(P)]for v in range(Q)])
    return lowpass

def highpass_filter(D,shape,filter_type='ideal'):
    Q, P = shape
    if filter_type == 'ideal':
        lowpass = np.array([[int((u - P//2)**2 + (v - Q//2)**2 > D**2) for u in range(P)] for v in range(Q)])
    else:
        lowpass = 1-lowpass_filter(D,shape,'gaussian')
    return lowpass

def apply_filter(im, f):
    N, M = im.shape
    im_padded = np.zeros((2*N, 2*M))
    im_padded[0:N,0:M] = im
    im_fft = np.fft.fft2(im_padded)
    im_fft = np.fft.fftshift(im_fft)
    res_fft = f * im_fft
    res = np.fft.ifft2(np.fft.ifftshift(res_fft))
    res = np.real(res)
    res = res[:N, :M]
    
    return res

H, W = background.shape
lowpass = lowpass_filter(50, (2*H,2*W), 'gauss')

diff_filtered = deepcopy(diff_threshold)

fig=plt.figure(figsize=(20, 40))

i = 1
for key in diff_threshold:
    plt.subplot(9, 4, i)
    plt.imshow(diff_threshold[key], cmap='gray')
    plt.title(key)
    plt.axis('off')
    plt.subplot(9, 4, i+1)
    plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(diff_threshold[key])))), cmap='gray')
    plt.axis('off')
    diff_filtered[key] = apply_filter(diff_threshold[key], lowpass)
    plt.subplot(9, 4, i+2)
    plt.imshow(diff_filtered[key], cmap='gray')
    plt.axis('off')
    plt.subplot(9, 4, i+3)
    plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(diff_filtered[key])))), cmap='gray')
    plt.axis('off')
    i += 4
        
plt.show()

On a alors éliminé du bruit, l'amélioration est par exemple particuliérement visible pour la première frame 

### Suppression manuelle

In [ ]:
fig=plt.figure(figsize=(10, 10))

diff_final = deepcopy(diff_filtered)

i = 1
for key in diff_enhanced:
    diff_final[key][:,:60] = 0 # on retire le trait vertical à gauche des images
    diff_final[key][-25:,:] = 0 # on retire le bas de l'outil
    plt.subplot(3, 3, i)
    plt.imshow(diff_final[key], cmap='gray')
    plt.axis('off')
    plt.title(key)
    i += 1

La ligne verticale à gauche de certaines des images a bien été éliminée.

### Erosion

In [ ]:
from skimage.morphology import dilation, erosion, opening, closing

fig=plt.figure(figsize=(10, 10))

diff_morph = deepcopy(diff_final)

i = 1
for key in diff_morph:
    diff_morph[key] = erosion(diff_morph[key], np.ones((4,4)))
    
    plt.subplot(3, 3, i)
    plt.imshow(diff_morph[key], cmap='gray')
    plt.axis('off')
    plt.title(key)
    i += 1

plt.show()

### Last thresholding

In [ ]:
fig=plt.figure(figsize=(10, 40))

result = deepcopy(diff_morph)
old = {}

i = 1
for key in diff_morph:
    result[key] = np.where(result[key] > 0.15 * np.max(result[key]), 0, 1)
    old[key] = np.where(diff_final[key] > 0.15 * np.max(diff_final[key]), 0, 1)
    plt.subplot(9, 2, i)
    plt.imshow(result[key], cmap='gray')
    plt.axis('off')
    plt.title(key)
    plt.subplot(9, 2, i+1)
    plt.imshow(old[key], cmap='gray')
    plt.axis('off')
    plt.title('old')
    i += 2

plt.show()

L'erosion a bien persmis de supprimer quelques points isolés de l'image et d'affiner le tracé

## Évaluation des performances

### Visualisation des masks

In [ ]:
IMDIR = 'catheter'
fig=plt.figure(figsize=(10, 40))

wire = {}
cath = {}

for root, dirnames, filenames in os.walk(IMDIR):
    for filename in filenames:
        f = os.path.join(root, filename)
        if filename.startswith(('2')):
            if not filename.startswith('201'):
                im = io.imread(f)
                if len(im.shape) > 2:
                    im = im[:,:,0]
                if filename.endswith('GuideWire.tiff'):
                    wire[filename[:3]] = im.astype('float') / 255
                else:
                    cath[filename[:3]] = im.astype('float') / 255

goal = deepcopy(wire)
for key in goal:
    goal[key] = (wire[key] + cath[key]) / 2
    
i = 1
for key in frames:
    plt.subplot(9, 2, i)
    plt.imshow(result[key], cmap='gray')
    plt.axis('off')
    plt.title(key)
    plt.subplot(9, 2, i+1)
    plt.imshow(goal[key], cmap='gray')
    plt.axis('off')
    plt.title(key)
    i += 2
    
plt.show()

### Calculs d'erreurs

lA FAIRE : partie 'Experimental' de la consigne

### Commentaires et conclusion

Lors de ce TP nous avons imaginé et mis en oeuvre une méthode permettant de suivre la trajectoire d'un outil sur différentes frames par rapport à un fond. Notre méthode s'appuie sur différentes notions du cours et permet d'obtenir des masks représentant assez clairement l'outil, de manière automatisée et identique pour les différentes frames.

En ce qui concerne la comparaison avec les masks objectifs fournis on remarque que notre méthode automatisée permet d'observer plus d'élements de l'outils (par exemple la partie gauche de celui-ci). Cependant le fil est moins visible que dans les masks fournis à cause de quelques discontinuités qui persistent et la présence d'élements qui certes appartiennent à l'outil mais ne sont pas vraiment pertinants pour répondre à la problématique (la partie inférieure de l'outil par exemple).

Ainsi, cette méthode présente un interêt pour des applications pratiques (gain de temps considérable par rapport à une méthode manuelle par exemple) et permettrait avec l'optimisation de certains paramètres ou l'utilisation de méthodes plus avancées d'obtenir en plus des résultats encore plus proches des masks fournis.